In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.plot import reshape_as_image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from sklearn.model_selection import train_test_split
import glob,os,sys,cv2
from datetime import datetime
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
training_path = "Input/sentinel/test_data_from_drive/patches_all/train/"
target_file_path = "Input/Target/concat/target_yield.shp"
patch_dim = (256, 256, 13)


In [16]:
BATCH_SIZE = 64
NUM_EPOCHS = 10
lr = 1e-4

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [3]:
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x

In [4]:
def read_training():
    training_file_list = glob.glob(os.path.join(training_path,"*.tif"))
    target_gdf = gpd.read_file(target_file_path)
    print("Total Number of Patches:",len(training_file_list))
    ignore_patch_list = list()
    x = list()
    y = list()
    X_train = list()
    X_test = list()
    y_train = list()
    y_test = list()
    count = 0 
    for file in training_file_list:

        patch_src = rio.open(file)
        f_name = file.split("/")[-1].split(".")[0]
        patch_src_read = reshape_as_image(patch_src.read()) ## Change the index here to add or remove the mask layer
        # print(0)
        if patch_src_read.shape != patch_dim:
            ignore_patch_list.append(f_name)
            # print("Patch Dimensions Mismatch, skipping patch : {}".format(f_name))
            continue

        # print(1)
        if np.isnan(patch_src_read).any():
            # print("Has Nan values, skipping patch : {}".format(f_name))
            continue

        # print(2)
        query = target_gdf.query(f"patch_name == '{f_name}'")["ykg_by_e7"]
        if len(query) != 1:
            # print("patch has no target value, skipping patch : {}".format(f_name))
            continue
        # print(patch_src_read[:,:,0:12].shape)
        # print(patch_src_read[:,:,12].shape)

        x.append(patch_src_read[:,:,0:12])
        y.append(patch_src_read[:,:,12])
        # y.append(float(query))

        patch_src.close()
        # print(count)
        count +=1
        if count > 100:
            break

    # self.y = self.scaler.fit_transform(np.array(self.y).reshape(-1, 1))
    y = np.array(y)
    y = np.expand_dims(y,-1)
    x = np.array(x)
    print("Any Null values? ",np.isnan(x).any())
    # print(self.y)
    # self.x = np.nan_to_num(self.x, nan=0)# Check for different value for no data
    print(f"x shape :{x.shape}, y shape: {y.shape}")
    # print(np.nanmin(self.x),np.nanmax(self.x))
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    return X_train, X_test, y_train, y_test
    #Also, split the training into train and val
    # For testing, keep a part of the dataset as seperate (final month)

X_train, X_test, y_train, y_test = read_training()

Total Number of Patches: 22707
Any Null values?  False
x shape :(101, 256, 256, 12), y shape: (101, 256, 256, 1)


In [5]:
# TRAIN_LENGTH = info.splits["train"].num_examples
STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE
VAL_SUBSPLITS = 5
# TEST_LENTH = info.splits["test"].num_examples
VALIDATION_STEPS = len(X_test) // BATCH_SIZE // VAL_SUBSPLITS

In [6]:
def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(256,256,12))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)

    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # outputs
    outputs = layers.Conv2D(1, 1, padding="same", activation = "softmax")(u9)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model


unet_model = build_unet_model()

2023-09-19 23:29:53.436180: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
def tf_parse(x,y):
    def f(x, y):
        return x, y
    images, masks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([256, 256, 12])
    masks.set_shape([256, 256, 1])
    return images, masks


def tf_dataset(x,y,batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    # break
    return dataset

train_dataset = tf_dataset(X_train,y_train,batch=BATCH_SIZE)
validation_dataset = tf_dataset(X_test,y_test,batch=BATCH_SIZE)

In [77]:
# for i,j in train_dataset:
#     i = i[0].numpy()
#     j = j[0].numpy()
#     j = np.squeeze(j,axis=-1)
#     print(i.shape,j.shape)
#     i=i*255
#     j = j*255.0
#     cv2.imwrite("img.png",i[:,:,3:6])
    
#     cv2.imwrite("mask.png",j)
#     # break

In [14]:
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]

unet_model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss="binary_crossentropy",
                  metrics=metrics)

callbacks = [
    ModelCheckpoint("unet_files/model_"+str(datetime.now())+".h5"),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=3),
    CSVLogger("unet_files/data_"+str(datetime.now())+".csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [15]:
model_history = unet_model.fit(train_dataset,
                              validation_data = validation_dataset,
                              epochs=NUM_EPOCHS,
                              callbacks=callbacks)
                              # steps_per_epoch=STEPS_PER_EPOCH,
                              # validation_steps=VALIDATION_STEPS)

Epoch 1/10
10/10 [==============================] - 60s 6s/step - loss: 0.5326 - acc: 0.2226 - recall_2: 1.0000 - precision_2: 0.2226 - iou: 0.2177 - val_loss: 0.5699 - val_acc: 0.2547 - val_recall_2: 1.0000 - val_precision_2: 0.2547 - val_iou: 0.2755 - lr: 1.0000e-04
Epoch 2/10
10/10 [==============================] - 58s 6s/step - loss: 0.5492 - acc: 0.2226 - recall_2: 1.0000 - precision_2: 0.2226 - iou: 0.2177 - val_loss: 0.5518 - val_acc: 0.2547 - val_recall_2: 1.0000 - val_precision_2: 0.2547 - val_iou: 0.2755 - lr: 1.0000e-04
Epoch 3/10
10/10 [==============================] - 58s 6s/step - loss: 0.5261 - acc: 0.2226 - recall_2: 1.0000 - precision_2: 0.2226 - iou: 0.2177 - val_loss: 0.5488 - val_acc: 0.2547 - val_recall_2: 1.0000 - val_precision_2: 0.2547 - val_iou: 0.2755 - lr: 1.0000e-04
Epoch 4/10
10/10 [==============================] - 57s 6s/step - loss: 0.5267 - acc: 0.2226 - recall_2: 1.0000 - precision_2: 0.2226 - iou: 0.2177 - val_loss: 0.5450 - val_acc: 0.2547 - val_re